# Héritage multiple

## Introduction

Dans le chapitre précédent de notre tutoriel, nous avons abordé l'héritage, ou plus précisément " l'héritage simple ". Comme nous l'avons vu, une classe hérite dans ce cas d'une seule classe. L'héritage multiple, quant à lui, est une fonctionnalité dans laquelle une classe peut hériter des attributs et des méthodes de plus d'une classe parente. Les critiques soulignent que l'héritage multiple s'accompagne d'un niveau élevé de complexité et d'ambiguïté dans des situations telles que le problème du diamant. Nous aborderons ce problème plus loin dans ce chapitre.

Le préjugé répandu selon lequel l'héritage multiple est quelque chose de "dangereux" ou de "mauvais" est principalement alimenté par des langages de programmation dont les mécanismes d'héritage multiple sont mal implémentés et surtout par une utilisation incorrecte de ceux-ci. Java ne supporte même pas l'héritage multiple, alors que C++ le supporte. Python a une approche sophistiquée et bien conçue de l'héritage multiple.

<center><img src="img/illustration11.png" width="70%"></center>

Une définition de classe, où une classe enfant ```SubClassName``` hérite des classes parents ```BaseClass1```, ```BaseClass2```, ```BaseClass3```, et ainsi de suite, ressemble à ceci :

```python
class SubclassName(BaseClass1, BaseClass2, BaseClass3, ...) :
    pass
```
Il est clair que toutes les superclasses ```BaseClass1```, ```BaseClass2```, ```BaseClass3```, ... peuvent également hériter d'autres superclasses. Ce que nous obtenons est un arbre d'héritage.

<center><img src="img/illustration12.png" width="70%"></center>

Exemple : ```CalendarClock```

Nous voulons introduire les principes de l'héritage multiple à l'aide d'un exemple. Pour cela, nous allons implémenter deux classes indépendantes : une classe ```Horloge``` et une classe ```Calendrier```. Ensuite, nous allons introduire une classe ```CalendarClock```, qui est, comme son nom l'indique, une combinaison de ```Clock``` et ```Calendar```. ```CalendarClock``` hérite à la fois de ```Clock``` et de ```Calendar```.

<center><img src="img/illustration13.png" width="70%"></center>

La classe ```Clock``` simule le tic-tac d'une horloge. Une instance de cette classe contient l'heure, qui est stockée dans les attributs ```self.hours```, ```self.minutes``` et ```self.seconds```. En principe, nous aurions pu écrire la méthode ```__init__``` et la méthode set comme ceci 

```python
def __init__(self,heures=0, minutes=0, secondes=0) :
        self._hours = heures
        self.__minutes = minutes
        self.__seconds = secondes
    def set(self,heures, minutes, secondes=0) :
        self._hours = heures
        self.__minutes = minutes
        self.__secondes = secondes
```   

Nous avons renoncé à cette implémentation, car nous avons ajouté du code supplémentaire pour vérifier la plausibilité des données temporelles dans la méthode set. Nous appelons également la méthode set à partir de la méthode ```__init__```, car nous voulons éviter le code redondant. 

La classe ```Clock``` complète :

In [1]:
""" 
The class Clock is used to simulate a clock.
"""
class Clock(object):
    
    def __init__(self, hours, minutes, seconds):
        """
        The paramaters hours, minutes and seconds have to be 
        integers and must satisfy the following equations:
        0 <= h < 24
        0 <= m < 60
        0 <= s < 60
        """
        self.set_Clock(hours, minutes, seconds)
        
    def set_Clock(self, hours, minutes, seconds):
        """
        The parameters hours, minutes and seconds have to be 
        integers and must satisfy the following equations:
        0 <= h < 24
        0 <= m < 60
        0 <= s < 60
        """
        if type(hours) == int and 0 <= hours and hours < 24:
            self._hours = hours
        else:
            raise TypeError("Hours have to be integers between 0 and 23!")
        if type(minutes) == int and 0 <= minutes and minutes < 60:
            self.__minutes = minutes 
        else:
            raise TypeError("Minutes have to be integers between 0 and 59!")
        if type(seconds) == int and 0 <= seconds and seconds < 60:
            self.__seconds = seconds
        else:
            raise TypeError("Seconds have to be integers between 0 and 59!")
            
    def __str__(self):
        return "{0:02d}:{1:02d}:{2:02d}".format(self._hours,
                                                self.__minutes,
                                                self.__seconds)
    
    def tick(self):
        """
        This method lets the clock "tick", this means that the 
        internal time will be advanced by one second.
        Examples:
        >>> x = Clock(12,59,59)
        >>> print(x)
        12:59:59
        >>> x.tick()
        >>> print(x)
        13:00:00
        >>> x.tick()
        >>> print(x)
        13:00:01
        """
        if self.__seconds == 59:
            self.__seconds = 0
            if self.__minutes == 59:
                self.__minutes = 0
                if self._hours == 23:
                    self._hours = 0
                else:
                    self._hours += 1
            else:
                self.__minutes += 1
        else:
            self.__seconds += 1
            
if __name__ == "__main__":
    x = Clock(23,59,59)
    print(x)
    x.tick()
    print(x)
    y = str(x)
    print(type(y))

23:59:59
00:00:00
<class 'str'>


Vérifions notre gestion des exceptions en entrant des flottants et des chaînes de caractères en entrée. Nous vérifions également ce qui se passe si nous dépassons les limites des valeurs attendues :

In [2]:
x = Clock(7.7, 45, 17)

TypeError: Hours have to be integers between 0 and 23!

In [3]:
x = Clock(24, 45, 17)

TypeError: Hours have to be integers between 0 and 23!

In [4]:
x = Clock(23, 60, 17)

TypeError: Minutes have to be integers between 0 and 59!

In [5]:
x = Clock("23", "60", "17")

TypeError: Hours have to be integers between 0 and 23!

In [6]:
x = Clock(23, 17)

TypeError: Clock.__init__() missing 1 required positional argument: 'seconds'

Nous allons maintenant créer une classe ```Calendar```, qui présente de nombreuses similitudes avec la classe ```Clock``` définie précédemment. Au lieu de ```tick```, nous avons une méthode ```advance```, qui fait avancer la date d'un jour, chaque fois qu'elle est appelée. L'ajout d'un jour à une date est assez délicat. Nous devons vérifier si la date est le dernier jour d'un mois et si le nombre de jours des mois varie. Comme si cela ne suffisait pas, nous avons le mois de février et le problème des années bissextiles.

Les règles de calcul d'une année bissextile sont les suivantes :

- Si une année est divisible par 400, c'est une année bissextile
- Si une année n'est pas divisible par 400 mais par 100, ce n'est pas une année bissextile.
- Un nombre d'année qui est divisible par 4 mais pas par 100, c'est une année bissextile.
- Tous les autres numéros d'année sont des années communes, c'est- -à-dire qu'il n'y a pas d'année bissextile.

Comme petite astuce utile, nous avons ajouté la possibilité d'afficher une date en style britannique ou américain (canadien).

In [7]:
""" 
The class Calendar implements a calendar.   
"""
class Calendar(object):
    months = (31,28,31,30,31,30,31,31,30,31,30,31)
    date_style = "British"
    
    @staticmethod
    def leapyear(year):
        """ 
        The method leapyear returns True if the parameter year
        is a leap year, False otherwise
        """
        if not year % 4 == 0:
            return False
        elif not year % 100 == 0:
            return True
        elif not year % 400 == 0:
            return False
        else:
            return True
        
    def __init__(self, d, m, y):
        """
        d, m, y have to be integer values and year has to be 
        a four digit year number
        """
        self.set_Calendar(d,m,y)
        
    def set_Calendar(self, d, m, y):
        """
        d, m, y have to be integer values and year has to be 
        a four digit year number
        """
        if type(d) == int and type(m) == int and type(y) == int:
            self.__days = d
            self.__months = m
            self.__years = y
        else:
            raise TypeError("d, m, y have to be integers!")
            
    def __str__(self):
        if Calendar.date_style == "British":
            return "{0:02d}/{1:02d}/{2:4d}".format(self.__days,
                                                   self.__months,
                                                   self.__years)
        else: 
            # assuming American style
            return "{0:02d}/{1:02d}/{2:4d}".format(self.__months,
                                                   self.__days,
                                                   self.__years)
    def advance(self):
        """
        This method advances to the next date.
        """
        max_days = Calendar.months[self.__months-1]
        if self.__months == 2 and Calendar.leapyear(self.__years):
            max_days += 1
        if self.__days == max_days:
            self.__days= 1
            if self.__months == 12:
                self.__months = 1
                self.__years += 1
            else:
                self.__months += 1
        else:
            self.__days += 1
            
if __name__ == "__main__":
    x = Calendar(31,12,2012)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("2012 was a leapyear:")
    x = Calendar(28,2,2012)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    x = Calendar(28,2,2013)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("1900 no leapyear: number divisible by 100 but not by 400: ")
    x = Calendar(28,2,1900)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("2000 was a leapyear, because number divisibe by 400: ")
    x = Calendar(28,2,2000)
    print(x, end=" ")
    x.advance()
    print("after applying advance: ", x)
    print("Switching to American date style: ")
    Calendar.date_style = "American"
    print("after applying advance: ", x)  

31/12/2012 after applying advance:  01/01/2013
2012 was a leapyear:
28/02/2012 after applying advance:  29/02/2012
28/02/2013 after applying advance:  01/03/2013
1900 no leapyear: number divisible by 100 but not by 400: 
28/02/1900 after applying advance:  01/03/1900
2000 was a leapyear, because number divisibe by 400: 
28/02/2000 after applying advance:  29/02/2000
Switching to American date style: 
after applying advance:  02/29/2000


Enfin, nous allons présenter notre exemple d'héritage multiple. Nous sommes maintenant capables d'implémenter la classe ```CalendarClock``` initialement prévue, qui héritera à la fois de ```Clock``` et de ```Calendar```. La méthode ```tick``` de ```Clock``` devra être surchargée. Cependant, la nouvelle méthode ```tick``` de ```CalendarClock``` doit appeler la méthode ```tick``` de ```Clock``` : ```Clock.tick(self)```

In [10]:
""" 
Module, which implements the class CalendarClock.
"""
from clock import Clock
from calendar import Calendar


class CalendarClock(Clock, Calendar):
    """ 
        The class CalendarClock implements a clock with integrated 
        calendar. It's a case of multiple inheritance, as it inherits 
        both from Clock and Calendar      
    """
    
    def __init__(self, day, month, year, hour, minute, second):
        Clock.__init__(self,hour, minute, second)
        Calendar.__init__(self, day, month, year)
        
    def tick(self):
        """
        advance the clock by one second
        """
        previous_hour = self._hours
        Clock.tick(self)
        if (self._hours < previous_hour): 
            self.advance()
            
    def __str__(self):
        return Calendar.__str__(self) + ", " + Clock.__str__(self)
    
    
if __name__ == "__main__":
    x = CalendarClock(31, 12, 2013, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)
    x = CalendarClock(28, 2, 1900, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)
    x = CalendarClock(28, 2, 2000, 23, 59, 59)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)
    x = CalendarClock(7, 2, 2013, 13, 55, 40)
    print("One tick from ",x, end=" ")
    x.tick()
    print("to ", x)

ModuleNotFoundError: No module named 'clock'

## Le problème du diamant ou le "diamant mortel de la mort"

<center><img src="img/illustration11.png" width="50%"></center>

Le "problème du diamant" (parfois appelé "diamant de la mort") est le terme généralement utilisé pour désigner une ambiguïté qui survient lorsque deux classes B et C héritent d'une superclasse A, et qu'une autre classe D hérite à la fois de B et de C. S'il existe une méthode "m" dans A que B ou C (ou même les deux) a surchargée, et si, en outre, il ne surcharge pas cette méthode, la question est alors de savoir de quelle version de la méthode D hérite. Il peut s'agir de celle de A, B ou C.

Prenons l'exemple de Python. La première configuration du problème du diamant est la suivante : B et C surchargent tous deux la méthode m de A :

In [11]:
class A:
    def m(self):
        print("m of A called")
class B(A):
    def m(self):
        print("m of B called")
class C(A):
    def m(self):
        print("m of C called")
class D(B,C):
    pass

Si on appelle la méthode m sur une instance x de D, c'est-à-dire x.m(), on obtiendra la sortie "m of B called". Si nous transposons l'ordre des classes dans l'en-tête de classe de D dans "class D(C,B) :", nous obtiendrons la sortie "m of C called".

Le cas où m ne sera surchargé que dans l'une des classes B ou C, par exemple dans C :

In [12]:
class A:
    def m(self):
        print("m of A called")
class B(A):
    pass
class C(A):
    def m(self):
        print("m of C called")
class D(B,C):
    pass
x = D()
x.m()

m of C called


Principalement, deux possibilités sont imaginables : On pourrait utiliser "m de C" ou "m de A".

Nous appelons ce script avec Python2.7 (python) et avec Python3 (python3) pour voir ce qui se passe :
```shell
$ python diamond1.py 
m de A appelé
$ python3 diamond1.py 
m de C appelé
```

Seulement pour ceux qui s'intéressent à la version 2 de Python : Pour avoir le même comportement d'héritage dans Python2 que dans Python3, chaque classe doit hériter de la classe ```object```. Notre classe A n'hérite pas d'object, nous obtenons donc une classe dite "à l'ancienne", si nous appelons le script avec python2. L'héritage multiple avec les classes à l'ancienne est régi par deux règles : la profondeur d'abord, puis la gauche à droite. Si vous changez la ligne d'en-tête de A en ```class A(object):```, nous aurons le même comportement dans les deux versions de Python.

```super()``` et MRO

Nous avons vu dans notre précédente implémentation du problème du diamant, comment Python résout le problème, c'est-à-dire dans quel ordre les classes de base sont parcourues. Cet ordre est défini par ce que l'on appelle le __Method Resolution Order__ ou en abrégé __MRO__.

Nous allons étendre notre exemple précédent, de sorte que chaque classe définisse sa propre méthode m :

In [13]:
class A:
    def m(self):
        print("m of A called")
class B(A):
    def m(self):
        print("m of B called")
class C(A):
    def m(self):
        print("m of C called")
class D(B,C):
    def m(self):
        print("m of D called")

Appliquons la méthode m sur une instance de D. Nous pouvons voir que seul le code de la méthode m de D sera exécuté. Nous pouvons également appeler explicitement les méthodes m des autres classes via le nom de la classe, comme nous le démontrons dans la session Python interactive suivante :

In [15]:
x = D()
B.m(x)

m of B called


In [16]:
C.m(x)

m of C called


In [17]:
A.m(x)

m of A called


Supposons maintenant que la méthode m de D doive également exécuter le code de m de B, C et A, lorsqu'elle est appelée. Nous pourrions l'implémenter comme suit :

In [18]:
class D(B,C):
    def m(self):
        print("m of D called")
        B.m(self)
        C.m(self)
        A.m(self)

Le résultat est celui que nous recherchions :

In [19]:
x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


Mais il s'avère une fois de plus que les choses sont plus compliquées qu'il n'y paraît. Comment pouvons-nous faire face à la situation, si m de B et m de C doivent appeler m de A également. Dans ce cas, nous devons supprimer l'appel ```A.m(self)``` de m dans D. Le code peut ressembler à ceci, mais il y a toujours un bug :

In [20]:
class A:
    def m(self):
        print("m of A called")
class B(A):
    def m(self):
        print("m of B called")
        A.m(self)
class C(A):
    def m(self):
        print("m of C called")
        A.m(self)
class D(B,C):
    def m(self):
        print("m of D called")
        B.m(self)
        C.m(self)

Le problème est que la méthode m de A sera appelée deux fois :

In [21]:
x = D()
x.m()

m of D called
m of B called
m of A called
m of C called
m of A called


Une façon de résoudre ce problème - certes non pythonique - consiste à scinder les méthodes m de B et C en deux méthodes. La première méthode, appelée ```_m```, consiste en le code spécifique de B et C et l'autre méthode s'appelle toujours m, mais consiste maintenant en un appel ```self._m()``` et un appel ```A.m(self)```. Le code de la méthode m de D consiste maintenant en le code spécifique de D ```print("m of D called"```, et les appels ```B._m(self)```, ```C._m(self)``` et ```A.m(self)``` :

In [22]:
class A:
    def m(self):
        print("m of A called")
        
class B(A):
    def _m(self):
        print("m of B called")
    def m(self):
        self._m()
        A.m(self)
        
class C(A):
    def _m(self):
        print("m of C called")
    def m(self):
        self._m()
        A.m(self)
        
        
class D(B,C):
    def m(self):
        print("m of D called")
        B._m(self)
        C._m(self)
        A.m(self)

Notre problème est résolu, mais - comme nous l'avons déjà mentionné - pas d'une manière pythonique :

In [23]:
x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


La façon optimale de résoudre le problème, qui est la super façon pythonique, serait d'appeler la ```super()``` fonction :

In [24]:
class A:
    def m(self):
        print("m of A called")
class B(A):
    def m(self):
        print("m of B called")
        super().m()
class C(A):
    def m(self):
        print("m of C called")
        super().m()
class D(B,C):
    def m(self):
        print("m of D called")
        super().m()

Il résout également notre problème, mais dans un design magnifique également :

In [25]:
x = D()
x.m()

m of D called
m of B called
m of C called
m of A called


La fonction ```super()``` est souvent utilisée lorsque les instances sont initialisées avec la méthode ```__init__``` :

In [26]:
class A:
    def __init__(self):
        print("A.__init__")
        
class B(A):
    def __init__(self):
        print("B.__init__")
        super().__init__()
        
class C(A):
    def __init__(self):
        print("C.__init__")
        super().__init__()
        
class D(B,C):
    def __init__(self):
        print("D.__init__")
        super().__init__()

Nous démontrons cette méthode de travail dans la session interactive suivante :

In [27]:
d = D()

D.__init__
B.__init__
C.__init__
A.__init__


In [28]:
c = C()

C.__init__
A.__init__


In [29]:
b = B()

B.__init__
A.__init__


In [30]:
a = A()

A.__init__


La question se pose de savoir comment les fonctions ```super()``` prennent leurs décisions. Comment décide-t-elle quelle classe doit être utilisée ? Comme nous l'avons déjà mentionné, elle utilise ce que l'on appelle la méthode de résolution d'ordre (MRO). Il est basé sur l'algorithme de "linéarisation de la superclasse C3". On parle de linéarisation, car l'arborescence est décomposée en un ordre linéaire. La méthode mro peut être utilisée pour créer cette liste :

In [31]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

In [32]:
B.mro()

[__main__.B, __main__.A, object]

In [33]:
A.mro()

[__main__.A, object]

## Polymorphisme

<center><img src="img/illustration14.png" width="50%"></center>

Le polymorphisme est construit à partir de deux mots grecs. "Poly" signifie "beaucoup" et "morph" signifie "forme". Le polymorphisme est l'état ou la condition d'être polymorphe, ou si l'on utilise la traduction des composants "la capacité d'être sous de nombreuses formes". Le polymorphisme est un terme utilisé dans de nombreux domaines scientifiques. En cristallographie, il définit l'état, si quelque chose se cristallise en deux ou plusieurs formes chimiquement identiques mais cristallographiquement distinctes. Les biologistes connaissent le polymorphisme comme l'existence d'un organisme en plusieurs variétés de formes ou de couleurs. Les Romains avaient même un dieu, appelé Morphée, qui est capable de prendre n'importe quelle forme humaine : Morphée apparaît dans les métamorphoses d'Ovide et est le fils de Somnus, le dieu du sommeil. Vous pouvez admirer Morphée et Iris sur la photo de droite.

Alors, avant de nous endormir, revenons à Python et à ce que signifie le polymorphisme dans le contexte des langages de programmation. Le polymorphisme en informatique est la capacité de présenter la même interface pour différentes formes sous-jacentes. Dans certains langages de programmation, nous pouvons avoir des fonctions ou des méthodes polymorphes, par exemple. Les fonctions ou méthodes polymorphes peuvent être appliquées à des arguments de différents types, et elles peuvent se comporter différemment selon le type des arguments auxquels elles sont appliquées. Nous pouvons également définir le même nom de fonction avec un nombre variable de paramètres.

Examinons la fonction Python suivante :

In [34]:
def f(x, y):
    print("values: ", x, y)
    
f(42, 43)
f(42, 43.7) 
f(42.3, 43)
f(42.0, 43.9)

values:  42 43
values:  42 43.7
values:  42.3 43
values:  42.0 43.9


Nous pouvons appeler cette fonction avec différents types, comme le montre l'exemple. Dans les langages de programmation typés comme Java ou C++, nous devrions surcharger f pour mettre en œuvre les différentes combinaisons de types.

Notre exemple pourrait être implémenté comme suit en C++ :

```c++
#include 
using namespace std;
void f(int x, int y ) {
    cout << "values: " << x << ", " << x << endl;
}
void f(int x, double y ) {
    cout << "values: " << x << ", " << x << endl;
}
void f(double x, int y ) {
    cout << "values: " << x << ", " << x << endl;
}
void f(double x, double y ) {
    cout << "values: " << x << ", " << x << endl;
}
int main()
{
    f(42, 43); 
    f(42, 43.7); 
    f(42.3,43);
    f(42.0, 43.9); 
}
```

Python est implicitement polymorphe. Nous pouvons appliquer notre fonction f précédemment définie même à des listes, des chaînes de caractères ou d'autres types, qui peuvent être imprimés :

In [36]:
def f(x,y):
    print("values: ", x, y)
    
f([3,5,6],(3,5))

values:  [3, 5, 6] (3, 5)


In [37]:
f("A String", ("A tuple", "with Strings"))

values:  A String ('A tuple', 'with Strings')
